Before running the code, connect to a T4 GPU and load the pdf files in content folder to read the text from the resilience files.

In [1]:
#Installing required libraries
!pip install --upgrade pip
!pip install farm-haystack[inference]==1.19.0
!pip install farm-haystack==1.19.0
!pip install sentence-transformers

!pip install PyMuPDF
!pip install flask-ngrok
!pip install requests beautifulsoup4
!pip install transformers torch pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[inference]==1.19.0 from https://files.pythonhosted.org/packages/df/ef/485cd648ee02afafd5c014b609c214299507112c246b75303f91fd2c139f/farm_haystack-1.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for canals==0.3.2 from https://files.pythonhosted.org/packages/b8/f6/6d2071a20400129a72390f021b46603f694d29553df0725152864c3c40f3/canals-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for prompthub-py==4.0.0 from https://files.pythonhosted.org/packages/27/5f/8c4939e290ff93af79364b88ffe3902d29c234

In [2]:
# Import necessary libraries
import os
import fitz
from haystack import Document
from haystack.document_stores import InMemoryDocumentStore
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.utils import convert_files_to_docs
from haystack.nodes import BM25Retriever
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok

# Create an instance of InMemoryDocumentStore for storing documents in memory
document_store = InMemoryDocumentStore(use_bm25=True)

# Import FARMReader from Haystack nodes for using a machine reading model
from haystack.nodes import FARMReader

# Import ExtractiveQAPipeline from Haystack pipelines for question-answering
from haystack.pipelines import ExtractiveQAPipeline

from haystack.utils import print_answers
from transformers import pipeline , RobertaTokenizer, RobertaForQuestionAnswering
import pdfplumber
import torch



In [3]:
# Path to the directory containing the PDF files
pdf_directory = "/content/"

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Iterate over PDF files in the directory
def read_all_files():
  for pdf_file in os.listdir(pdf_directory):
      if pdf_file.endswith(".pdf"):
          pdf_path = os.path.join(pdf_directory, pdf_file)
          text = extract_text_from_pdf(pdf_path)
          # Create a Document object for each PDF using the Document constructor provided by Haystack
          doc = Document(content=text, meta={"name": pdf_file})
          # Write the Document to the InMemoryDocumentStore
          document_store.write_documents([doc])

# Call the function to read and process all PDF files in the specified directory

In [4]:
# Read and process all PDF files in the specified directory to extract text and store as Haystack Documents
pdf_text = read_all_files()
# Initialize a BM25Retriever using the configured InMemoryDocumentStore
retriever = BM25Retriever(document_store=document_store)
# Initialize an FARMReader for question-answering using the specified pre-trained model
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
# Create an ExtractiveQAPipeline by combining the FARMReader and BM25Retriever
pipe = ExtractiveQAPipeline(reader, retriever)


Updating BM25 representation...: 100%|██████████| 1/1 [00:00<00:00, 72.94 docs/s]


In [5]:
# Define the question for which you want to find answers
question = "What is major concern of Paris"

# Use the ExtractiveQAPipeline to run the question through the configured components
# Retrieve relevant documents using the retriever (top_k: 10) and then find answers using the reader (top_k: 5)
prediction = pipe.run(query = question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

# Print the predicted answers extracted from the documents
print_answers(prediction)

Inferencing Samples: 100%|██████████| 8/8 [00:09<00:00,  1.18s/ Batches]

'Query: What is major concern iof Paris'
'Answers:'
[   <Answer {'answer': 'Governance', 'type': 'extractive', 'score': 0.8647494912147522, 'context': 'operation between \nterritories that are increasingly interdependent. \nGovernance is therefore a priority challenge within \nthis resilience strategy.\nE', 'offsets_in_document': [{'start': 42491, 'end': 42501}], 'offsets_in_context': [{'start': 70, 'end': 80}], 'document_ids': ['d737d94e882b78af83b96242f1793870'], 'meta': {'name': 'Paris-Resilience-Strategy-English.pdf'}}>,
    <Answer {'answer': 'climate change', 'type': 'extractive', 'score': 0.8576041460037231, 'context': ' have to evolve to make the City \nmore responsive to the effects of climate change.\nA first step would be to identify the current regulatory \nobstacle', 'offsets_in_document': [{'start': 122038, 'end': 122052}], 'offsets_in_context': [{'start': 68, 'end': 82}], 'document_ids': ['d737d94e882b78af83b96242f1793870'], 'meta': {'name': 'Paris-Resilience-Strategy-En

Inferencing Samples: 100%|██████████| 8/8 [00:05<00:00,  1.43 Batches/s]


In [18]:
# Define the question for which you want to find answers
question = "Is Crime rate high in Pune??"

# Use the ExtractiveQAPipeline to run the question through the configured components
# Retrieve relevant documents using the retriever (top_k: 10) and then find answers
prediction = pipe.run(query = question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

# Print the predicted answers extracted from the documents, showing only minimum details
print_answers(prediction, details="minimum")

Inferencing Samples: 100%|██████████| 14/14 [00:10<00:00,  1.40 Batches/s]

'Query: Is Crime rate high in Pune??'
'Answers:'
[   {   'answer': 'Pune faces disease outbreak threats',
        'context': 'ty, dependencies and in developing a \n'
                   'sense of belonging.\n'
                   'Pune faces disease outbreak threats due to mismanagement '
                   'of \n'
                   'its biodiversity. Pune’s climat'},
    {   'answer': 'lack of public understanding and support delays the \n'
                  'transition to a sustainable mobility system in Pune',
        'context': 'ng mobility \n'
                   'services, lack of public understanding and support delays '
                   'the \n'
                   'transition to a sustainable mobility system in Pune. '
                   'Protection \n'
                   'of pedest'},
    {   'answer': 'there is a lack of an overarching, responsive and '
                  'proactive \n'
                  'environmental governance framework',
        'context': 'vernance in Pu